In [2]:
import cv2
import numpy as np
import tensorflow as tf
#yolo model load
yolonet=cv2.dnn.readNet('yoloface-master/model-weights/yolov3-wider_16000.weights','yoloface-master/cfg/yolov3-face.cfg')

In [2]:
import tensorflow as tf
googlenet=tf.keras.models.load_model('cat_soft_bi.h5')
#googlenet model load

In [3]:
layer_names = yolonet.getLayerNames()
output_layers = [layer_names[i - 1] for i in yolonet.getUnconnectedOutLayers()]

In [10]:
#실행하는 하드웨어마다 카메라 size가 다르므로 설정해줌
cap = cv2.VideoCapture(0)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
channel=3

#bounding box color임의 지정 (2,3)으로 2가지 색깔 가져옴(helmet_on,helmet_off)
colors=np.random.uniform(0,255,size=(2,3)) 

on_col=colors[0] #helmet on color
off_col=colors[1] #helmet off color
font = cv2.FONT_HERSHEY_PLAIN

print(width,height)

1280 720


In [ ]:
cv2.startWindowThread()
cap = cv2.VideoCapture(0)

while(True):
    # reading the frame
    ret, frame = cap.read()
    
    # preprocessing for yolo
    blob=cv2.dnn.blobFromImage(frame,0.00392,(416,416),(0,0,0),True,crop=False) #yolo 적용을 위한 전처리
    yolonet.setInput(blob)
    outs=yolonet.forward(output_layers)
    
    boxes=[]
    confidences=[]
    class_ids=[]
    for out in outs:
        for detection in out:
            confidence=detection[5] #class face로 1개이므로 [5:]가 아닌 5만 보면 됨
            if confidence>0.1: #정확도를 위해서 올릴 수 있어
                center_x=int(detection[0]*width)
                center_y=int(detection[1]*height)
                w=int(detection[2]*width)
                h=int(detection[3]*height)

                x=int(center_x-w/2)
                y=int(center_y-h/2)

                boxes.append([x,y,w,h]) #후보군들
                confidences.append(float(confidence))
    
    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.1,0.4) #boxes로 선정된 것 중 Confidence가 threshold이상인 것들만 indexes에

    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h=boxes[i]
            #helmet size까지 고려하여 bbox size 수정 & Frame 밖으로 나가서 오류나지 않도록 min,max
            x_1=np.max([0,x-int(0.4*w)])
            y_1=np.max([0,y-int(0.7*h)])
            x_2=np.min([width,x+int(1.4*w)])
            y_2=np.min([height,y+int(1.2*h)])
            croped=frame[y_1:y_2,x_1:x_2]
            croped=cv2.resize(croped,(224,224))
            croped=croped.reshape(1,224,224,3)
            prob=model.predict(croped,verbose=False)[0] #return 3개(output, aux1, aux2)
            pred=np.argmax(prob)
            if pred==0: #pred==0 : 헬멧착용
                cv2.rectangle(frame,(x_1,y_1),(x_2,y_2),on_col,2)
                cv2.putText(frame,'on',(x_1,y_1+5),font,1,on_col,2)
            else: #pred==1 : 헬멧미착용
                cv2.rectangle(frame,(x_1,y_1),(x_2,y_2),off_col,2)
                cv2.putText(frame,'off',(x_1,y_1+5),font,1,off_col,2)
            

    cv2.imshow('frame',frame)
        
    if cv2.waitKey(1)==ord('q'):
        # breaking the loop if the user types q
        # note that the video window must be highlighted!
        break


cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)